In [ ]:
!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install transformers==4.35.0
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install evaluate
!pip install -qqq trl==0.7.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 3.0 MB/s eta 0:00:00


In [ ]:
# %%capture
# %pip install -U bitsandbytes
%pip install -U transformers
# %pip install -U peft
%pip install -U accelerate
# %pip install -U trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 18.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch,warnings
from datasets import load_dataset
from trl import SFTTrainer
os.environ["WANDB_DISABLED"] = "true"
warnings.filterwarnings("ignore")

In [ ]:
base_model = "mistralai/Mistral-7B-v0.1" #include the model
dataset_name = "mlabonne/guanaco-llama2-1k"
new_model = "mistral_7b_gua/1" #give your own name

In [ ]:
pip install pandas

In [ ]:
import pandas as pd
df = pd.read_csv("/content/final_dataset_Finetuning.csv")
df = df.iloc[:,2:]
df.columns = ["whisper AI corrected",'Reference']
df = df[~df['whisper AI corrected'].isnull()]
df.tail()

,whisper AI corrected,Reference
85,"Hi, I am Dr. D. V. L. Narayan Rao, Senior Con...","Hi, I am Dr. D. V. L. Narayan Rao, a Senior Co..."
86,"I'm Dr. D. V. L. Normandra, Senior Consultant...","I'm Dr. D. V. L. Normandra, Senior Consultant,..."
87,"Hi, I am Dr. A Mohan Rao, Senior Consultant, ...","""Hi, I am Dr. A Mohan Rao, a Senior Consultant..."
88,"Hi, I am Dr. Mohan Rao, Senior Consultant, Ge...","Hi, I am Dr. Mohan Rao, a Senior Consultant in..."
89,"Hi, I am Dr. Mohan Rao, Senior Consultant, Ge...","Hi, I am Dr. Mohan Rao, a Senior Consultant in..."


In [ ]:
import json
result = list(df.to_json(orient="records"))

result[0] = '{"json":['
result[-1] = ']'
result.append('}')


result = ''.join(result)
result = result.strip('"\'')

result = json.loads(result)

with open('data.json', 'w') as json_file:
    json.dump(result, json_file)

In [ ]:
dataset = load_dataset("json", data_files="data.json", field='json', split="train")
def format_instruction(Original_Paragraph: str, Corrected_Paragraph: str):
	return f"""<s>[INST]Correct the Paragraph- {Original_Paragraph.strip()}[/INST]
'{Corrected_Paragraph}'
""".strip()
def process_dataset(data: dataset):
    return (
        data.shuffle(seed=42)
        .map(generate_instruction_dataset)
    )


def generate_instruction_dataset(data_point):

    return {
        "Original Paragraph": data_point["Original Paragraph"],
        "Corrected Paragraph": data_point["Corrected Paragraph"],
        "text": format_instruction(data_point["Original Paragraph"],data_point["Corrected Paragraph"])
    }
dataset = process_dataset(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/88 [00:00<?, ? examples/s]

In [ ]:
pip install sentencepiece

In [ ]:
pip install --upgrade transformers tokenizers

In [ ]:
# Load base model(Mistral 7B)

bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

model = AutoModelForCausalLM.from_pretrained(
        pretrained_model_name_or_path=base_model,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# import sentencepiece
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

(True, True)

In [ ]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [ ]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=5,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= 1024,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

Map:   0%|          | 0/88 [00:00<?, ? examples/s]

In [ ]:
# wandb.finish()
trainer.train()

Step,Training Loss
5,1.522600
10,1.199200


Step,Training Loss
5,1.522600
10,1.199200
15,1.117200
20,1.101000
25,1.073000
30,0.929200
35,0.905000
40,0.808700
45,0.900200


In [ ]:
trainer.model.save_pretrained(new_model)
# wandb.finish()
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Lin

In [ ]:
try:
    trainer.model.push_to_hub(new_model, use_temp_dir=False)
except:
    print("An exception occurred")

In [ ]:
input_string = " " #Enter your input

In [ ]:
logging.set_verbosity(logging.CRITICAL)

text = ''

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=700)
result = pipe(f"<s>[INST]Correct the Paragraph- {input_string} [/INST]")
res = result[0]['generated_text'].split('\n')[0][1:-1]
print(res)
